In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# add venv PATH to shell command PATH
import sys, os
if sys.base_prefix not in os.environ['PATH']:
    os.environ['PATH'] = f"{sys.base_prefix}/bin:{os.environ['PATH']}"

In [5]:
NAME = 'linear_sleep_mlflow'

# load data & train model(not used, just placeholder)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
rf = RandomForestRegressor(n_estimators=num_estimators)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.67 0.   1.24 1.   0.   1.   2.   1.   0.   2.
 0.   1.93 2.   2.   0.   0.   1.   2.   1.   1.31 1.48 1.93 1.   1.
 2.   2.  ]
  mse: 0.092760


In [7]:
import pickle
tmpdir = 'mlflow_tmp'
data_path = os.path.join(tmpdir, 'skmodel.pkl')
with open(data_path, 'wb') as of:
    pickle.dump(rf, of)

In [14]:
%%writefile {NAME}.py
from __future__ import print_function

import os
import pickle

import pandas as pd
import numpy as np
import pytest
import six
import time


import mlflow
import mlflow.pyfunc
import mlflow.pyfunc.model
from mlflow.models import Model


def _load_pyfunc(path):
    with open(path, 'rb') as of:
        data_model = pickle.load(of)
    class Model:
        def predict(self, inputs):
            a, b = inputs.to_numpy()[0]
            x = inputs.shape[0]
            time.sleep(a * x + b)
            return inputs
    return Model()
        


if __name__ == '__main__':
    tmpdir = 'mlflow_tmp'
    data_path = os.path.join(tmpdir, 'skmodel.pkl')
    model_path = os.path.join(str(tmpdir), "model")

    model_config = Model(run_id="test")
    mlflow.pyfunc.save_model(path=model_path,
                             data_path=data_path,
                             loader_module=os.path.basename(__file__)[:-3],
                             code_path=[__file__],
                             mlflow_model=model_config)


Overwriting linear_sleep_mlflow.py


In [23]:
!rm -r {tmpdir}/model
!python {NAME}.py
!mlflow models serve -m {tmpdir}/model

/opt/anaconda3/envs/bentoml-dev-py36/lib/python3.6/site-packages/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/anaconda3/envs/bentoml-dev-py36/lib/python3.6/site-packages/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2020/03/26 17:48:17 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2020/03/26 17:48:17 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2020-03-26 17:48:17 +0800] [3262602] [INFO] Starting gunicorn 20.0.4
[2020-03-26 17:48:17 +0800] [3262602] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2020-03-26 17:48:17 +0800] [3262602] [ERROR] Retrying in 1 second.
[2020-03-26 17:48:18 +0800] [3262602] [INF

# Test with requests

In [20]:
import json
import requests
import pandas as pd
import numpy as np

headers = {"content-type": "application/json"}
raw_data = np.array([[0, 2]])
data = pd.DataFrame(raw_data,
                    columns=map(str, range(raw_data.shape[1]))).to_json(orient='split')

json_response = requests.post(f'http://127.0.0.1:5000/invocations',
                              data=data, headers=headers)
print(json_response)
print(json_response.json())

<Response [200]>
[{'0': 0, '1': 2}]


# Benchmark with locust

In [24]:
%%writefile benchmark_{NAME}.py
from locust import HttpLocust, TaskSet, task, constant
from functools import lru_cache

import pandas as pd
import random
import numpy as np
import json


import os

A = float(os.environ.get('A', 0))
B = float(os.environ.get('B', 0))
WAIT = float(os.environ.get('WAIT', 1))



@lru_cache(maxsize=1)
def data_producer():

    headers = {"content-type": "application/json"}

    def _gen_data():
        _A = A + random.random() / 100000.0
        _B = B + random.random() / 10000.0

        raw_data = np.array([[_A, _B]])
        data = pd.DataFrame(raw_data,columns=map(str, range(raw_data.shape[1]))).to_json(orient='split')
        return headers, data

    return _gen_data


class WebsiteTasks(TaskSet):

    @task
    def index(self):
        headers, data = data_producer()()
        self.client.post("/invocations", data, headers=headers)

class WebsiteUser(HttpLocust):
    task_set = WebsiteTasks
    wait_time = constant(WAIT)

Overwriting benchmark_linear_sleep_mlflow.py


In [22]:
!locust -f benchmark_{NAME}.py -H http://127.0.0.1:5000

[2020-03-26 17:47:23,562] beta/INFO/locust.main: Starting web monitor at http://*:8089
[2020-03-26 17:47:23,562] beta/INFO/locust.main: Starting Locust 0.14.4
[2020-03-26 17:47:46,973] beta/INFO/locust.runners: Hatching and swarming 10 users at the rate 1 users/s (0 users already running)...
[2020-03-26 17:47:55,983] beta/INFO/locust.runners: All locusts hatched: WebsiteUser: 10 (0 already running)
^C
[2020-03-26 17:48:08,198] beta/ERROR/stderr: KeyboardInterrupt
[2020-03-26 17:48:08,198] beta/ERROR/stderr: 2020-03-26T09:48:08Z
[2020-03-26 17:48:08,198] beta/ERROR/stderr: 
[2020-03-26 17:48:08,198] beta/INFO/locust.main: Shutting down (exit code 0), bye.
[2020-03-26 17:48:08,198] beta/INFO/locust.main: Cleaning up runner...
[2020-03-26 17:48:08,199] beta/INFO/locust.main: Running teardowns...
 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s failures/s
-----------------------------------------------------------